# 📊 Notebook 1: Master DataFrame Builder

## 🎯 Purpose
Build a comprehensive master dataframe containing ALL required columns for subsequent analysis.

## 📋 Output
- **File**: `master_inventory_data.csv`
- **File**: `master_inventory_data.pkl` (for faster loading)
- **Used by**: Notebooks 2, 3, 4, 5

## 🔑 Key Columns
1. **Item Info**: item_code, item_name, item_group, creation_date
2. **Stock Data**: current_stock, reserved_stock, available_stock, stock_value
3. **Sales History**: total_qty_sold, total_sales_value, last_sale_date
4. **Time Periods**: qty_7d, qty_14d, qty_30d, qty_60d, qty_90d, qty_180d
5. **Transactions**: transaction_count, unique_customers, avg_transaction_qty
6. **Calculated Metrics**: days_since_creation, days_since_last_sale, velocity, turnover
7. **Variability**: std_dev, cv, cv_squared
8. **Financial**: avg_price, total_revenue, profit_margin

In [3]:
# 📚 Import Libraries
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries imported successfully")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries imported successfully
Analysis Date: 2026-01-13 10:42:50


## 1️⃣ Database Connection

In [4]:
# Database credentials
user = 'root'
password = '2001'
host = 'localhost'
database = '_f77958810e04f2ae'

connection_string = f'mysql+pymysql://{user}:{password}@{host}/{database}'

try:
    engine = create_engine(connection_string)
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1"))
        result.fetchone()
    print("✅ Database connection successful")
except Exception as e:
    print(f"❌ Connection failed: {e}")

✅ Database connection successful


## 2️⃣ Extract Complete Data from ERPNext

### Comprehensive Query for ALL Required Columns

In [5]:
query = """
WITH ItemBasics AS (
    SELECT 
        i.item_code,
        i.item_name,
        i.item_group,
        i.creation as creation_date,
        DATEDIFF(CURDATE(), i.creation) as days_since_creation,
        i.standard_rate as standard_price,
        i.valuation_rate,
        i.is_stock_item,
        i.disabled
    FROM `tabItem` i
    WHERE i.disabled = 0 
        AND i.is_stock_item = 1
),
SalesHistory AS (
    SELECT 
        sii.item_code,
        -- First and last sale
        MIN(si.posting_date) as first_sale_date,
        MAX(si.posting_date) as last_sale_date,
        DATEDIFF(CURDATE(), MAX(si.posting_date)) as days_since_last_sale,
        
        -- Transaction counts
        COUNT(DISTINCT si.name) as transaction_count,
        COUNT(DISTINCT si.customer) as unique_customers,
        
        -- All time totals
        SUM(sii.qty) as total_qty_sold,
        SUM(sii.amount) as total_sales_value,
        AVG(sii.qty) as avg_transaction_qty,
        STDDEV(sii.qty) as stddev_transaction_qty,
        AVG(sii.rate) as avg_selling_price,
        
        -- Last 7 days
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 7 DAY) 
            THEN sii.qty ELSE 0 END) as qty_7d,
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 7 DAY) 
            THEN sii.amount ELSE 0 END) as sales_7d,
        COUNT(DISTINCT CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 7 DAY) 
            THEN si.name END) as transactions_7d,
        
        -- Last 14 days
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 14 DAY) 
            THEN sii.qty ELSE 0 END) as qty_14d,
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 14 DAY) 
            THEN sii.amount ELSE 0 END) as sales_14d,
            
        -- Last 30 days
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY) 
            THEN sii.qty ELSE 0 END) as qty_30d,
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY) 
            THEN sii.amount ELSE 0 END) as sales_30d,
        COUNT(DISTINCT CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY) 
            THEN si.name END) as transactions_30d,
        
        -- Last 60 days
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 60 DAY) 
            THEN sii.qty ELSE 0 END) as qty_60d,
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 60 DAY) 
            THEN sii.amount ELSE 0 END) as sales_60d,
            
        -- Last 90 days
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 90 DAY) 
            THEN sii.qty ELSE 0 END) as qty_90d,
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 90 DAY) 
            THEN sii.amount ELSE 0 END) as sales_90d,
        COUNT(DISTINCT CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 90 DAY) 
            THEN si.name END) as transactions_90d,
        
        -- Last 180 days
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 180 DAY) 
            THEN sii.qty ELSE 0 END) as qty_180d,
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 180 DAY) 
            THEN sii.amount ELSE 0 END) as sales_180d,
        COUNT(DISTINCT CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 180 DAY) 
            THEN si.name END) as transactions_180d,
            
        -- Last 365 days
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 365 DAY) 
            THEN sii.qty ELSE 0 END) as qty_365d,
        SUM(CASE WHEN si.posting_date >= DATE_SUB(CURDATE(), INTERVAL 365 DAY) 
            THEN sii.amount ELSE 0 END) as sales_365d
            
    FROM `tabSales Invoice Item` sii
    INNER JOIN `tabSales Invoice` si ON sii.parent = si.name
    WHERE si.docstatus = 1
    GROUP BY sii.item_code
),
CurrentStock AS (
    SELECT 
        item_code,
        SUM(actual_qty) as current_stock,
        SUM(reserved_qty) as reserved_stock,
        SUM(ordered_qty) as ordered_stock,
        SUM(stock_value) as stock_value,
        AVG(valuation_rate) as avg_valuation_rate,
        COUNT(DISTINCT warehouse) as warehouse_count
    FROM `tabBin`
    WHERE actual_qty > 0 OR reserved_qty > 0
    GROUP BY item_code
),
PurchaseData AS (
    SELECT 
        pri.item_code,
        COUNT(DISTINCT pr.name) as purchase_count,
        SUM(pri.qty) as total_purchased_qty,
        MAX(pr.posting_date) as last_purchase_date,
        AVG(pri.rate) as avg_purchase_rate,
        AVG(DATEDIFF(pr.posting_date, pr.creation)) as avg_lead_time_days
    FROM `tabPurchase Receipt Item` pri
    INNER JOIN `tabPurchase Receipt` pr ON pri.parent = pr.name
    WHERE pr.docstatus = 1
        AND pr.posting_date >= DATE_SUB(CURDATE(), INTERVAL 365 DAY)
    GROUP BY pri.item_code
)
SELECT 
    -- Item Info
    ib.item_code,
    ib.item_name,
    ib.item_group,
    ib.creation_date,
    ib.days_since_creation,
    ib.standard_price,
    ib.valuation_rate,
    
    -- Stock Data
    COALESCE(cs.current_stock, 0) as current_stock,
    COALESCE(cs.reserved_stock, 0) as reserved_stock,
    COALESCE(cs.ordered_stock, 0) as ordered_stock,
    COALESCE(cs.current_stock - cs.reserved_stock, 0) as available_stock,
    COALESCE(cs.stock_value, 0) as stock_value,
    COALESCE(cs.avg_valuation_rate, ib.valuation_rate, 0) as avg_valuation_rate,
    COALESCE(cs.warehouse_count, 0) as warehouse_count,
    
    -- Sales History
    sh.first_sale_date,
    sh.last_sale_date,
    COALESCE(sh.days_since_last_sale, 999) as days_since_last_sale,
    COALESCE(sh.transaction_count, 0) as transaction_count,
    COALESCE(sh.unique_customers, 0) as unique_customers,
    COALESCE(sh.total_qty_sold, 0) as total_qty_sold,
    COALESCE(sh.total_sales_value, 0) as total_sales_value,
    COALESCE(sh.avg_transaction_qty, 0) as avg_transaction_qty,
    COALESCE(sh.stddev_transaction_qty, 0) as stddev_transaction_qty,
    COALESCE(sh.avg_selling_price, 0) as avg_selling_price,
    
    -- Time period sales
    COALESCE(sh.qty_7d, 0) as qty_7d,
    COALESCE(sh.sales_7d, 0) as sales_7d,
    COALESCE(sh.transactions_7d, 0) as transactions_7d,
    
    COALESCE(sh.qty_14d, 0) as qty_14d,
    COALESCE(sh.sales_14d, 0) as sales_14d,
    
    COALESCE(sh.qty_30d, 0) as qty_30d,
    COALESCE(sh.sales_30d, 0) as sales_30d,
    COALESCE(sh.transactions_30d, 0) as transactions_30d,
    
    COALESCE(sh.qty_60d, 0) as qty_60d,
    COALESCE(sh.sales_60d, 0) as sales_60d,
    
    COALESCE(sh.qty_90d, 0) as qty_90d,
    COALESCE(sh.sales_90d, 0) as sales_90d,
    COALESCE(sh.transactions_90d, 0) as transactions_90d,
    
    COALESCE(sh.qty_180d, 0) as qty_180d,
    COALESCE(sh.sales_180d, 0) as sales_180d,
    COALESCE(sh.transactions_180d, 0) as transactions_180d,
    
    COALESCE(sh.qty_365d, 0) as qty_365d,
    COALESCE(sh.sales_365d, 0) as sales_365d,
    
    -- Purchase Data
    COALESCE(pd.purchase_count, 0) as purchase_count,
    COALESCE(pd.total_purchased_qty, 0) as total_purchased_qty,
    pd.last_purchase_date,
    COALESCE(pd.avg_purchase_rate, 0) as avg_purchase_rate,
    COALESCE(pd.avg_lead_time_days, 7) as avg_lead_time_days
    
FROM ItemBasics ib
LEFT JOIN SalesHistory sh ON ib.item_code = sh.item_code
LEFT JOIN CurrentStock cs ON ib.item_code = cs.item_code
LEFT JOIN PurchaseData pd ON ib.item_code = pd.item_code
ORDER BY ib.item_code
"""

print("⏳ Extracting data from ERPNext...")
print("   This may take 2-5 minutes for large datasets...\n")

df = pd.read_sql(query, engine)

print(f"✅ Data extracted successfully")
print(f"   Total items: {len(df):,}")
print(f"   Total columns: {len(df.columns)}")

⏳ Extracting data from ERPNext...
   This may take 2-5 minutes for large datasets...

✅ Data extracted successfully
   Total items: 13,489
   Total columns: 47


In [6]:
df

,item_code,item_name,item_group,creation_date,days_since_creation,standard_price,valuation_rate,current_stock,reserved_stock,ordered_stock,available_stock,stock_value,avg_valuation_rate,warehouse_count,first_sale_date,last_sale_date,days_since_last_sale,transaction_count,unique_customers,total_qty_sold,total_sales_value,avg_transaction_qty,stddev_transaction_qty,avg_selling_price,qty_7d,sales_7d,transactions_7d,qty_14d,sales_14d,qty_30d,sales_30d,transactions_30d,qty_60d,sales_60d,qty_90d,sales_90d,transactions_90d,qty_180d,sales_180d,transactions_180d,qty_365d,sales_365d,purchase_count,total_purchased_qty,last_purchase_date,avg_purchase_rate,avg_lead_time_days
0,10010000100001,EATON PISTON PUMP (02-142405) PVH98,AXIAL PISTON PUMP,2021-11-09 18:09:35.105322,1526,15000.00,10000.00,0.00,0.00,0.00,0.00,0.00,10000.00,0,2021-11-13,2021-11-13,1522,1,1,1.00,3993.00,1.00,0.00,3993.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,None,0.00,7.00
1,11000001,HD PP TUBE CLAMP 48.3MM,CLAMP,2022-08-01 17:21:48.859728,1261,0.00,0.00,2.00,0.00,0.00,2.00,58.00,29.00,1,None,None,999,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,None,0.00,7.00
2,11000002,PARKER HD PP TUBE CLAMP 33.7MM,CLAMP,2022-08-19 15:55:56.439134,1243,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,None,None,999,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,None,0.00,7.00
3,11000003,TUBE CLAMP HDPP 48.30MM -RCP2-448.3 - PARKER,CLAMP,2022-08-19 16:00:30.511292,1243,0.00,24.31,0.00,0.00,0.00,0.00,0.00,24.31,0,2024-06-29,2025-02-28,319,2,2,25.00,482.40,8.33,3.40,27.75,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,20.00,132.40,0,0.00,None,0.00,7.00
4,11000004,PARKER HD PP TUBE CLAMP 26.9MM,CLAMP,2022-08-19 16:27:59.022391,1243,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,None,None,999,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,None,0.00,7.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13484,WR2-0061,WR2 PU OTH WS 150.00 X 162.00 X 05.50/10.00,WR2 PU,2024-01-04 10:19:09.728261,740,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,None,None,999,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,None,0.00,7.00
13485,WR2-0062,WR2 PU OTH WS 155.00 X 167.00 X 05.50/10.00,WR2 PU,2024-01-04 10:19:52.598639,740,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,None,None,999,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,None,0.00,7.00
13486,WR2-0063,WR2 PU OTH WS 160.00 X 172.00 X 05.50/10.00,WR2 PU,2024-01-04 10:20:37.600939,740,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,None,None,999,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,None,0.00,7.00
13487,WR2-0064,WR2 PU OTH WS 175.00 X 187.00 X 05.50/10.00,WR2 PU,2024-01-04 10:21:14.241181,740,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,None,None,999,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,None,0.00,7.00


### 📦 Inventory Analytics Master Dataset — Column Dictionary

This dataset is generated from ERPNext and provides a complete analytical view of inventory, sales, stock, and procurement.

Each column is documented with:
- Meaning (what the value represents)
- Formula (if derived)
- Business purpose (how it is used)

---

#### 🧱 Item Master Information

| Column | Meaning | Formula / Source | Business Purpose |
|--------|--------|-----------------|----------------|
| `item_code` | Unique item identifier | Item master key | Primary key for analysis |
| `item_name` | Item display name | Item master | Human readable item |
| `item_group` | Item category | Item master | Category-level analysis |
| `creation_date` | Item creation date | Item master | Identify new vs old items |
| `days_since_creation` | Item age in days | `CURDATE() - creation_date` | New item performance tracking |
| `standard_price` | Default selling price | Item master | Price benchmarking |
| `valuation_rate` | Default cost price | Item master | Cost valuation |

---

#### 📦 Stock & Warehouse Information

| Column | Meaning | Formula / Source | Business Purpose |
|--------|--------|-----------------|----------------|
| `current_stock` | Total stock across warehouses | `SUM(actual_qty)` | Inventory on hand |
| `reserved_stock` | Stock reserved for orders | `SUM(reserved_qty)` | Committed demand |
| `ordered_stock` | Stock ordered from suppliers | `SUM(ordered_qty)` | Incoming stock |
| `available_stock` | Stock free to sell | `current_stock - reserved_stock` | Sellable inventory |
| `stock_value` | Inventory financial value | `SUM(stock_value)` | Cash locked in stock |
| `avg_valuation_rate` | Avg cost across warehouses | `AVG(valuation_rate)` | True cost price |
| `warehouse_count` | Warehouses holding item | `COUNT(DISTINCT warehouse)` | Distribution footprint |

---

#### 📊 Lifetime Sales Performance

| Column | Meaning | Formula / Source | Business Purpose |
|--------|--------|-----------------|----------------|
| `first_sale_date` | First sales date | `MIN(posting_date)` | Product adoption |
| `last_sale_date` | Latest sales date | `MAX(posting_date)` | Sales recency |
| `days_since_last_sale` | Days since last sale | `CURDATE() - last_sale_date` | Dead stock detection |
| `transaction_count` | Total invoices | `COUNT(DISTINCT invoice)` | Sales frequency |
| `unique_customers` | Number of buyers | `COUNT(DISTINCT customer)` | Market reach |
| `total_qty_sold` | Lifetime quantity sold | `SUM(qty)` | Demand strength |
| `total_sales_value` | Lifetime revenue | `SUM(amount)` | Business contribution |
| `avg_transaction_qty` | Avg qty per sale | `AVG(qty)` | Order behavior |
| `stddev_transaction_qty` | Variation in qty | `STDDEV(qty)` | Demand stability |
| `avg_selling_price` | Avg selling price | `AVG(rate)` | Pricing performance |

---

#### ⏱ Short-Term Demand Signals

##### Last 7 Days

| Column | Meaning | Formula | Business Purpose |
|--------|--------|---------|----------------|
| `qty_7d` | Qty sold last 7 days | `SUM(qty WHERE date>=today-7)` | Weekly velocity |
| `sales_7d` | Revenue last 7 days | `SUM(amount WHERE date>=today-7)` | Weekly revenue |
| `transactions_7d` | Orders last 7 days | `COUNT(invoice WHERE date>=today-7)` | Weekly demand |

---

##### Last 14 Days

| Column | Meaning | Formula | Business Purpose |
|--------|--------|---------|----------------|
| `qty_14d` | Qty sold last 14 days | `SUM(qty WHERE date>=today-14)` | Fortnight trend |
| `sales_14d` | Revenue last 14 days | `SUM(amount WHERE date>=today-14)` | Demand momentum |

---

##### Last 30 Days

| Column | Meaning | Formula | Business Purpose |
|--------|--------|---------|----------------|
| `qty_30d` | Qty sold last 30 days | `SUM(qty WHERE date>=today-30)` | Monthly velocity |
| `sales_30d` | Revenue last 30 days | `SUM(amount WHERE date>=today-30)` | Performance |
| `transactions_30d` | Orders last 30 days | `COUNT(invoice WHERE date>=today-30)` | Order frequency |

---

##### Last 60 Days

| Column | Meaning | Formula | Business Purpose |
|--------|--------|---------|----------------|
| `qty_60d` | Qty sold last 60 days | `SUM(qty WHERE date>=today-60)` | Medium-term demand |
| `sales_60d` | Revenue last 60 days | `SUM(amount WHERE date>=today-60)` | Growth trend |

---

##### Last 90 Days

| Column | Meaning | Formula | Business Purpose |
|--------|--------|---------|----------------|
| `qty_90d` | Qty sold last 90 days | `SUM(qty WHERE date>=today-90)` | Quarterly demand |
| `sales_90d` | Revenue last 90 days | `SUM(amount WHERE date>=today-90)` | Business contribution |
| `transactions_90d` | Orders last 90 days | `COUNT(invoice WHERE date>=today-90)` | Sales stability |

---

##### Last 180 Days

| Column | Meaning | Formula | Business Purpose |
|--------|--------|---------|----------------|
| `qty_180d` | Qty sold last 180 days | `SUM(qty WHERE date>=today-180)` | Seasonal demand |
| `sales_180d` | Revenue last 180 days | `SUM(amount WHERE date>=today-180)` | Half-year performance |
| `transactions_180d` | Orders last 180 days | `COUNT(invoice WHERE date>=today-180)` | Stability |

---

##### Last 365 Days

| Column | Meaning | Formula | Business Purpose |
|--------|--------|---------|----------------|
| `qty_365d` | Qty sold last 1 year | `SUM(qty WHERE date>=today-365)` | Annual demand |
| `sales_365d` | Revenue last 1 year | `SUM(amount WHERE date>=today-365)` | Annual business impact |

---

#### 🛒 Purchase & Supply Chain Information

| Column | Meaning | Formula / Source | Business Purpose |
|--------|--------|-----------------|----------------|
| `purchase_count` | No. of purchase receipts | `COUNT(purchase_receipts)` | Procurement frequency |
| `total_purchased_qty` | Total purchased qty (1 year) | `SUM(purchase_qty)` | Supply volume |
| `last_purchase_date` | Most recent purchase date | `MAX(posting_date)` | Reorder recency |
| `avg_purchase_rate` | Avg buying cost | `AVG(purchase_rate)` | Supplier pricing |
| `avg_lead_time_days` | Avg supplier lead time | `AVG(posting_date - creation)` | Reorder planning |

---

#### 🎯 Core Business Metrics Enabled

| Metric | Formula |
|--------|--------|
| Avg Daily Sales | `qty_90d / 90` |
| Stock Cover Days | `current_stock / avg_daily_sales` |
| Reorder Point | `avg_daily_sales × avg_lead_time_days` |
| Dead Stock Flag | `days_since_last_sale > 180 AND current_stock > 0` |
| New Item Flag | `days_since_creation < 60` |
| Fast Moving Flag | `qty_30d high AND transactions_30d high` |

---

**Total Columns: 47**

This is your Inventory Analytics Master Table.


## 3️⃣ Calculate Derived Metrics

### Add calculated columns for analysis

In [5]:
print("⏳ Calculating derived metrics...\n")

# 1. Velocity metrics (units per day)
df['velocity_7d'] = df['qty_7d'] / 7
df['velocity_30d'] = df['qty_30d'] / 30
df['velocity_90d'] = df['qty_90d'] / 90
df['velocity_180d'] = df['qty_180d'] / 180

# 2. Days since first activity
df['first_activity_date'] = df['first_sale_date'].fillna(df['creation_date'])
df['days_since_first_activity'] = (
    pd.to_datetime('today') - pd.to_datetime(df['first_activity_date'])
).dt.days

# 3. Days of stock (how long current stock will last)
df['days_of_stock'] = np.where(
    df['velocity_90d'] > 0,
    df['current_stock'] / df['velocity_90d'],
    999
)

# 4. Stock turnover (times per year)
df['annual_turnover'] = np.where(
    df['current_stock'] > 0,
    (df['qty_365d'] / df['current_stock']),
    0
)

# 5. Coefficient of Variation (CV)
df['cv'] = np.where(
    df['avg_transaction_qty'] > 0,
    df['stddev_transaction_qty'] / df['avg_transaction_qty'],
    0
)

# 6. CV² (for SBC classification)
df['cv_squared'] = df['cv'] ** 2

# 7. Average Demand Interval (ADI) - days between sales
df['adi'] = np.where(
    df['transaction_count'] > 0,
    df['days_since_first_activity'] / df['transaction_count'],
    999
)

# 8. Transaction frequency (transactions per day)
df['transaction_frequency'] = np.where(
    df['days_since_first_activity'] > 0,
    df['transaction_count'] / df['days_since_first_activity'],
    0
)

# 9. Customer concentration (customers per transaction)
df['customer_concentration'] = np.where(
    df['transaction_count'] > 0,
    df['unique_customers'] / df['transaction_count'],
    0
)

# 10. Revenue per unit
df['revenue_per_unit'] = np.where(
    df['total_qty_sold'] > 0,
    df['total_sales_value'] / df['total_qty_sold'],
    df['standard_price']
)

# 11. Profit margin estimate
df['profit_margin_pct'] = np.where(
    df['avg_selling_price'] > 0,
    ((df['avg_selling_price'] - df['avg_valuation_rate']) / df['avg_selling_price'] * 100),
    0
)

# 12. Velocity trend (30d vs 90d)
df['velocity_trend'] = np.where(
    df['velocity_90d'] > 0,
    df['velocity_30d'] / df['velocity_90d'],
    1.0
)

# 13. Growth rate (comparing periods)
df['growth_30d_vs_90d'] = np.where(
    (df['qty_90d'] - df['qty_30d']) > 0,
    ((df['qty_30d'] / 30) / ((df['qty_90d'] - df['qty_30d']) / 60) - 1) * 100,
    0
)

# 14. Has recent sales flag
df['has_sales_7d'] = df['qty_7d'] > 0
df['has_sales_30d'] = df['qty_30d'] > 0
df['has_sales_90d'] = df['qty_90d'] > 0
df['has_any_sales'] = df['total_qty_sold'] > 0

# 15. Stock status flags
df['has_stock'] = df['current_stock'] > 0
df['has_reserved'] = df['reserved_stock'] > 0
df['has_ordered'] = df['ordered_stock'] > 0

print("✅ Calculated metrics:")
print("   - Velocity metrics (7d, 30d, 90d, 180d)")
print("   - Days of stock & turnover")
print("   - Variability (CV, CV², ADI)")
print("   - Financial (profit margin, revenue per unit)")
print("   - Trends (velocity trend, growth rate)")
print("   - Boolean flags (has_sales, has_stock)")

⏳ Calculating derived metrics...

✅ Calculated metrics:
   - Velocity metrics (7d, 30d, 90d, 180d)
   - Days of stock & turnover
   - Variability (CV, CV², ADI)
   - Financial (profit margin, revenue per unit)
   - Trends (velocity trend, growth rate)
   - Boolean flags (has_sales, has_stock)


## 4️⃣ Data Quality Check

In [6]:
print("\n📊 DATA QUALITY REPORT\n")
print("="*80)

# Basic info
print(f"\n1. Dataset Size:")
print(f"   Rows: {len(df):,}")
print(f"   Columns: {len(df.columns)}")
print(f"   Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Missing values
print(f"\n2. Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing': missing[missing > 0],
    'Percentage': missing_pct[missing > 0]
}).sort_values('Missing', ascending=False)

if len(missing_df) > 0:
    print(missing_df.head(10))
else:
    print("   ✅ No missing values!")

# Data distribution
print(f"\n3. Sales Distribution:")
print(f"   Items with sales: {df['has_any_sales'].sum():,} ({df['has_any_sales'].mean()*100:.1f}%)")
print(f"   Items without sales: {(~df['has_any_sales']).sum():,} ({(~df['has_any_sales']).mean()*100:.1f}%)")

print(f"\n4. Stock Distribution:")
print(f"   Items with stock: {df['has_stock'].sum():,} ({df['has_stock'].mean()*100:.1f}%)")
print(f"   Items without stock: {(~df['has_stock']).sum():,} ({(~df['has_stock']).mean()*100:.1f}%)")

print(f"\n5. Key Metrics Summary:")
summary_cols = ['current_stock', 'stock_value', 'total_qty_sold', 'total_sales_value',
                'velocity_30d', 'days_of_stock', 'transaction_count']
print(df[summary_cols].describe())

print("\n" + "="*80)


📊 DATA QUALITY REPORT


1. Dataset Size:
   Rows: 13,489
   Columns: 71
   Memory: 11.00 MB

2. Missing Values:
                    Missing  Percentage
last_purchase_date    12352       91.57
first_sale_date        9812       72.74
last_sale_date         9812       72.74

3. Sales Distribution:
   Items with sales: 3,673 (27.2%)
   Items without sales: 9,816 (72.8%)

4. Stock Distribution:
   Items with stock: 2,986 (22.1%)
   Items without stock: 10,503 (77.9%)

5. Key Metrics Summary:
       current_stock  stock_value  total_qty_sold  total_sales_value  \
count       13489.00     13489.00        13489.00           13489.00   
mean            6.73       227.57           14.30            1002.50   
std            43.50      3201.18          165.44           11064.36   
min             0.00         0.00            0.00               0.00   
25%             0.00         0.00            0.00               0.00   
50%             0.00         0.00            0.00               0.00   
75%

## 5️⃣ Save Master DataFrame

In [7]:
# Save as CSV
csv_path = '/tmp/master_inventory_data.csv'
df.to_csv(csv_path, index=False)
print(f"✅ Saved CSV: {csv_path}")


# Save as Pickle (faster loading)
pkl_path = '/tmp/master_inventory_data.pkl'
df.to_pickle(pkl_path)
print(f"✅ Saved Pickle: {pkl_path}")


# Save column list
with open('/tmp/master_columns.txt', 'w') as f:
    f.write('\n'.join(df.columns.tolist()))
print(f"✅ Saved column list: /tmp/master_columns.txt")

print(f"\n📊 MASTER DATAFRAME SUMMARY:")
print(f"   Total Columns: {len(df.columns)}")
print(f"   Total Rows: {len(df):,}")
print(f"\n🎯 Ready for Notebooks 2, 3, 4, 5")

✅ Saved CSV: /tmp/master_inventory_data.csv
✅ Saved Pickle: /tmp/master_inventory_data.pkl
✅ Saved column list: /tmp/master_columns.txt

📊 MASTER DATAFRAME SUMMARY:
   Total Columns: 71
   Total Rows: 13,489

🎯 Ready for Notebooks 2, 3, 4, 5


## 6️⃣ Column Reference Guide

In [8]:
# Group columns by category
column_groups = {
    'Item Info': [
        'item_code', 'item_name', 'item_group', 'creation_date', 'days_since_creation',
        'standard_price', 'valuation_rate'
    ],
    'Stock Data': [
        'current_stock', 'reserved_stock', 'ordered_stock', 'available_stock',
        'stock_value', 'avg_valuation_rate', 'warehouse_count'
    ],
    'Sales History': [
        'first_sale_date', 'last_sale_date', 'days_since_last_sale', 'days_since_first_activity',
        'transaction_count', 'unique_customers', 'total_qty_sold', 'total_sales_value',
        'avg_transaction_qty', 'stddev_transaction_qty', 'avg_selling_price'
    ],
    'Time Periods': [
        'qty_7d', 'qty_14d', 'qty_30d', 'qty_60d', 'qty_90d', 'qty_180d', 'qty_365d',
        'sales_7d', 'sales_14d', 'sales_30d', 'sales_60d', 'sales_90d', 'sales_180d', 'sales_365d',
        'transactions_7d', 'transactions_30d', 'transactions_90d', 'transactions_180d'
    ],
    'Velocity Metrics': [
        'velocity_7d', 'velocity_30d', 'velocity_90d', 'velocity_180d',
        'days_of_stock', 'annual_turnover', 'velocity_trend'
    ],
    'Variability': [
        'cv', 'cv_squared', 'adi', 'transaction_frequency', 'customer_concentration'
    ],
    'Financial': [
        'revenue_per_unit', 'profit_margin_pct', 'growth_30d_vs_90d'
    ],
    'Purchase Data': [
        'purchase_count', 'total_purchased_qty', 'last_purchase_date',
        'avg_purchase_rate', 'avg_lead_time_days'
    ],
    'Boolean Flags': [
        'has_sales_7d', 'has_sales_30d', 'has_sales_90d', 'has_any_sales',
        'has_stock', 'has_reserved', 'has_ordered'
    ]
}

print("\n📋 COLUMN REFERENCE GUIDE\n")
print("="*80)

total_cols = 0
for category, cols in column_groups.items():
    total_cols += len(cols)
    print(f"\n{category} ({len(cols)} columns):")
    for col in cols:
        dtype = df[col].dtype if col in df.columns else 'N/A'
        print(f"   - {col:30s} ({dtype})")

print(f"\n" + "="*80)
print(f"Total columns documented: {total_cols}")
print(f"Total columns in dataframe: {len(df.columns)}")

if total_cols != len(df.columns):
    missing_cols = set(df.columns) - set([c for cats in column_groups.values() for c in cats])
    if missing_cols:
        print(f"\n⚠️ Undocumented columns: {missing_cols}")


📋 COLUMN REFERENCE GUIDE


Item Info (7 columns):
   - item_code                      (object)
   - item_name                      (object)
   - item_group                     (object)
   - creation_date                  (datetime64[ns])
   - days_since_creation            (int64)
   - standard_price                 (float64)
   - valuation_rate                 (float64)

Stock Data (7 columns):
   - current_stock                  (float64)
   - reserved_stock                 (float64)
   - ordered_stock                  (float64)
   - available_stock                (float64)
   - stock_value                    (float64)
   - avg_valuation_rate             (float64)
   - warehouse_count                (int64)

Sales History (11 columns):
   - first_sale_date                (object)
   - last_sale_date                 (object)
   - days_since_last_sale           (int64)
   - days_since_first_activity      (int64)
   - transaction_count              (int64)
   - unique_customers        

## ✅ Summary

### Files Created:
1. **`master_inventory_data.csv`** - Complete dataset (use in Notebooks 2-5)
2. **`master_inventory_data.pkl`** - Faster loading format
3. **`master_columns.txt`** - Column reference

### Next Steps:
Run **Notebook 2** to perform DBSCAN clustering for dead stock and new items identification.